<a href="https://colab.research.google.com/github/karjain/Anti_deepFake/blob/main/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
%cd /content/drive

Mounted at /content/drive
/content/drive


In [3]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
#import tensorflow_hub as hub

import os
import argparse
from os.path import join
import cv2
import dlib
#import torch
#import torch.nn as nn
from PIL import Image as pil_image
from tqdm import tqdm

In [4]:
!ls

sample_data


In [116]:
reload_model = tf.keras.models.load_model('/content/drive/MyDrive/Fake_dect/modelK.h5')

In [78]:
def get_boundingbox(face, width, height, scale=1.3, minsize=None):
    """
    Expects a dlib face to generate a quadratic bounding box.
    :param face: dlib face class
    :param width: frame width
    :param height: frame height
    :param scale: bounding box size multiplier to get a bigger face region
    :param minsize: set minimum bounding box size
    :return: x, y, bounding_box_size in opencv form
    """
    x1 = face.left()
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    size_bb = int(max(x2 - x1, y2 - y1) * scale)
    if minsize:
        if size_bb < minsize:
            size_bb = minsize
    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

    # Check for out of bounds, x-y top left corner
    x1 = max(int(center_x - size_bb // 2), 0)
    y1 = max(int(center_y - size_bb // 2), 0)
    # Check for too big bb size for given x, y
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)

    return x1, y1, size_bb

In [62]:
def video_to_croped_frames(video_files_path, output_path, frame_rate=33 ):


  os.makedirs(output_path+'/test', exist_ok=True)
    
  videos_processed=0

  reader = cv2.VideoCapture(video_files_path)
  
  video_name = video_files_path.split('/')[-1]
  video_fn = video_name.split('.')[0]

  # Face detector
  face_detector = dlib.get_frontal_face_detector()
  
  # Frame numbers and length of output video
  i=0
  c=1
  while reader.isOpened():
      _, image = reader.read()
      if image is None:
          break
      if c%frame_rate==1:
  
        # Image size
        height, width = image.shape[:2]

        # . Detect with dlib
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_detector(gray, 1)
        if len(faces):
            for face in faces:
              # Face crop with dlib and bounding box scale enlargement
              x, y, size = get_boundingbox(face, width, height)
              cropped_face = image[y:y+size, x:x+size]
              print(output_path + "/test/" + video_fn+str(i) + '.jpg')
              cv2.imwrite(str(output_path + "/test/" + video_fn+str(i) + '.jpg'), cropped_face)
              i+=1 
      c+=1           
        
  print(video_name)


In [70]:
!rm /content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/*.jpg

In [71]:
#video_to_croped_frames('/content/drive/MyDrive/Fake_dect/adohdulfwb.mp4','/content/drive/MyDrive/Fake_dect/Trial_Dataset/test')
#video_to_croped_frames('/content/drive/MyDrive/Fake_dect/abarnvbtwb.mp4','/content/drive/MyDrive/Fake_dect/Trial_Dataset/test')

video_to_croped_frames('/content/drive/MyDrive/Fake_dect/aagfhgtpmv.mp4','/content/drive/MyDrive/Fake_dect/Trial_Dataset/test')

/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/aagfhgtpmv0.jpg
/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/aagfhgtpmv1.jpg
/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/aagfhgtpmv2.jpg
/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/aagfhgtpmv3.jpg
/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/aagfhgtpmv4.jpg
/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/aagfhgtpmv5.jpg
/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/aagfhgtpmv6.jpg
/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/aagfhgtpmv7.jpg
/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/aagfhgtpmv8.jpg
/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/test/aagfhgtpmv9.jpg
aagfhgtpmv.mp4


In [72]:
def create_ds(path, img_shape):
  valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
  str(path),
  validation_split=0,
  seed=123,
  label_mode=None,
  image_size=(img_shape[0], img_shape[1]),
  batch_size=8)
  
  normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
  flip_layer = tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal')
  contrast_layer = tf.keras.layers.experimental.preprocessing.RandomContrast(factor=(0,2))
  
  valid_ds = valid_ds.map(lambda x: (normalization_layer(x)))
  valid_ds = valid_ds.map(lambda x: (flip_layer(x)))
  valid_ds = valid_ds.map(lambda x: (contrast_layer(x)))
  
  #for image in valid_ds:
  #  predictions = reload_model.predict(image)
  #  print(predictions)
  return valid_ds

In [73]:
img_shape = [224,224]
valid_ds = create_ds('/content/drive/MyDrive/Fake_dect/Trial_Dataset/test/',img_shape) 
#valid_ds = create_ds('/content/drive/MyDrive/Fake_dect/Trial_Dataset/real',img_shape) 

Found 10 files belonging to 1 classes.


In [ ]:
for image in valid_ds:
    predictions = reload_model.predict(image)
    print(predictions)